In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

import missingno as msno
from tqdm import tqdm_notebook

In [2]:
train_iden = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\train_identity.csv")
train_tran = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\train_transaction.csv")
test_iden = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\test_identity.csv")
test_tran = pd.read_csv(r"C:\Users\EPOPCON\Desktop\Kaggle\test_transaction.csv")

In [3]:
print (len (train_iden))
print (len (train_tran))
print (len (test_iden))
print (len (test_tran))

144233
590540
141907
506691


In [4]:
train_sample = train_tran[['isFraud','V280',
       'V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(train_sample.mean())

In [5]:
train_sample = train_tran[['isFraud','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 
       'C8', 'C9', 'C10', 'C11','C12', 'C13', 'C14','D1',
                           
       'V95','V96','V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
       'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114',
       'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123',
       'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132','V133', 'V134', 'V135', 'V136','V137',
                           
       'V280','V284', 'V285', 'V286', 'V287', 'V290', 'V291', 'V292',
       'V293', 'V294', 'V295', 'V297', 'V298', 'V299', 'V303', 'V304', 'V305', 'V306']]
sample = train_sample.fillna(train_sample.mean())

In [6]:
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import * 

In [7]:
import imblearn.under_sampling as usam
import imblearn.over_sampling as osam 
from sklearn.ensemble import RandomForestClassifier


X = sample.drop('isFraud', axis = 1)
Y = sample['isFraud']
feature_names = X.columns.tolist() # 컬럼을 리스트화
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, test_size = 0.1, random_state = 13,
                                                    shuffle= False)

rus = usam.RandomUnderSampler()
undersampled_X, undersampled_Y = rus.fit_sample(Train_X, Train_Y)
model = RandomForestClassifier(n_estimators = 200, max_depth = 10, random_state = 13).fit(undersampled_X, undersampled_Y)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.8053810538625781


In [8]:
df_undersampled_X = pd.DataFrame(columns = feature_names,data = undersampled_X)
df_undersampled_Y = pd.DataFrame(columns = ['isFraud'], data = undersampled_Y)

In [9]:
df_under_sample = pd.concat([df_undersampled_Y,df_undersampled_X],axis = 1)

In [10]:
# 특징을 전부 적용했을 때의 정확도 계산
X = df_under_sample.drop('isFraud', axis = 1)
Y = df_under_sample['isFraud']
feature_names = X.columns.tolist() # 컬럼을 리스트화

Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
model = RandomForestClassifier(n_estimators = 200, max_depth = 10, random_state = 13)
model.fit(Train_X, Train_Y)
predicted = model.predict(Test_X)
print (roc_auc_score(Test_Y, predicted))

0.7924921863891771


In [ ]:
Number_of_better_feature_set = 0
for L in range(2, len(feature_names)): # 2부터 feature -1 개수만큼 for문을 반복
    for subset in tqdm_notebook(itertools.combinations(feature_names, L)): # feature_names에서 L개를 선택하는 모든 조합(튜플)
        New_Train_X = Train_X[list(subset)] # X[tuple]은 작동하지 않아서, X[list]형태로 바꿔준 것
        New_Test_X = Test_X[list(subset)]        
        model = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
        model.fit(New_Train_X, Train_Y) 
        predicted = model.predict(New_Test_X)
        New_accuracy = roc_auc_score(Test_Y, predicted)
        if New_accuracy > 0.81:
            Number_of_better_feature_set += 1
            print(list(subset), '\t', len(subset), '\t', New_accuracy) 

In [ ]:
# 수정본
Number_of_better_feature_set = 0
len_ = len(feature_names)
for L in tqdm_notebook(range(len_,1,-1)): # 2부터 feature -1 개수만큼 for문을 반복
    for subset in itertools.combinations(feature_names, L): # feature_names에서 L개를 선택하는 모든 조합(튜플)
        New_Train_X = Train_X[list(subset)] # X[tuple]은 작동하지 않아서, X[list]형태로 바꿔준 것
        New_Test_X = Test_X[list(subset)]        
        model = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
        model.fit(New_Train_X, Train_Y) 
        predicted = model.predict(New_Test_X)
        New_accuracy = roc_auc_score(Test_Y, predicted)
        if New_accuracy > 0.81:
            Number_of_better_feature_set += 1
            print(list(subset), '\t', len(subset), '\t', New_accuracy) 